In [4]:
%pip install aioarango
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
from aioarango import ArangoClient
import os
from dotenv import load_dotenv

load_dotenv()

username_arango = os.getenv('arango_username')
password_arango = os.getenv('arango_password')
database_arango = os.getenv('arango_database')
sys_username_arango = os.getenv('sys_username_arango')
sys_password_arango = os.getenv('sys_password_arango')

print(username_arango)
print(password_arango)
print(database_arango)

# Initialize the client for ArangoDB.
client = ArangoClient(hosts="https://arangodb.roulet.dev")

# Connect to "_system" database as root user.
sys_db = await client.db("_system", username=str(sys_username_arango), password=str(sys_password_arango))

# Create a new database named "DDI_MC2".
try:
  await sys_db.create_database("DDI_MC2")
except Exception as e:
  print(e)

# Connect to "test" database as root user.
db = await client.db(str(database_arango), username=str(username_arango), password=str(password_arango))

# Create a new collection named "students".
try:
  users = await db.create_collection("users")
except Exception as e:
  print(e)
  
try:
  jwt_tokens = await db.create_collection("jwt_tokens")
except Exception as e:
  print(e)
  
try:
  movies = await db.create_collection("movies")
except Exception as e:
  print(e)

await users.add_hash_index(fields=["name"], unique=True)

user = [{"name": "John", "age": 25}, {"name": "Jane", "age": 30}, {"name": "Jack", "age": 20}, {"name": "Jill", "age": 20}, {"name": "Jenny", "age": 20}, {"name": "Jenny", "age": 20}]

for u in user:
  try:
    await users.insert(u)
  except Exception as e:
    print(e)



  

DDI2
Welcome12
DDI_MC2
[HTTP 409][ERR 1207] duplicate database name 'DDI_MC2'
[HTTP 409][ERR 1207] duplicate name
[HTTP 409][ERR 1207] duplicate name
[HTTP 409][ERR 1207] duplicate name


c:\Users\super\AppData\Local\Programs\Python\Python310\lib\site-packages\httpx\_client.py:1978: UserWarning: Unclosed <httpx.AsyncClient object at 0x000002035F1A69B0>. See https://www.python-httpx.org/async/#opening-and-closing-clients for details.
  warnings.warn(


NameError: name 'name' is not defined